<h1>Without GUI</h1>

In [7]:
import yt_dlp

def download_youtube_video(video_url, download_path='E:\\Python\\pythonProject\\downloads'):
    ydl_opts = {
        'outtmpl': f'{download_path}/%(title)s.%(ext)s',
        'format': 'best',
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

if __name__ == "__main__":
    video_url = input("Enter the YouTube video URL: ")
    download_path = "E:\\Python\\pythonProject\\downloads"
    download_youtube_video(video_url, download_path)

[youtube] Extracting URL: https://youtu.be/bsfXmaGQBeM?si=O8sLf5P65y0SPYw8
[youtube] bsfXmaGQBeM: Downloading webpage
[youtube] bsfXmaGQBeM: Downloading tv client config
[youtube] bsfXmaGQBeM: Downloading player 6b3caec8
[youtube] bsfXmaGQBeM: Downloading tv player API JSON
[youtube] bsfXmaGQBeM: Downloading ios player API JSON
[youtube] bsfXmaGQBeM: Downloading m3u8 information
[info] bsfXmaGQBeM: Downloading 1 format(s): 18
[download] Destination: E:\Python\pythonProject\downloads\Cool Countdown 10,9,8,7,6,5,4,3,2,1.mp4
[download] 100% of  991.06KiB in 00:00:07 at 141.46KiB/s 


<h1>With  GUI (1)</h1>

In [8]:
import tkinter as tk
from tkinter import ttk, scrolledtext, filedialog, messagebox
import threading
import os
import yt_dlp
from tkinter.font import Font
from tkinter import PhotoImage
import re

class YouTubeDownloaderGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("YouTube Video Downloader")
        self.root.geometry("800x600")
        self.root.configure(bg="#2C3E50")  # Dark blue background
        
        # Default download path
        self.download_path = "E:\\Python\\pythonProject\\downloads"
        
        # Create custom fonts
        self.title_font = Font(family="Arial", size=16, weight="bold")
        self.button_font = Font(family="Arial", size=12, weight="bold")
        self.text_font = Font(family="Arial", size=10)
        
        self.create_widgets()
        
    def create_widgets(self):
        # Main frame
        main_frame = tk.Frame(self.root, bg="#2C3E50", padx=20, pady=20)
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Title
        title_label = tk.Label(
            main_frame, 
            text="YouTube Video Downloader", 
            font=self.title_font, 
            bg="#2C3E50", 
            fg="#ECF0F1"  # Light gray text
        )
        title_label.pack(pady=(0, 20))
        
        # URL input frame
        url_frame = tk.Frame(main_frame, bg="#34495E", padx=10, pady=10, relief=tk.RAISED, bd=1)
        url_frame.pack(fill=tk.BOTH, expand=True, pady=(0, 10))
        
        url_label = tk.Label(
            url_frame, 
            text="Enter YouTube URLs (one per line):", 
            font=self.text_font, 
            bg="#34495E", 
            fg="#ECF0F1"
        )
        url_label.pack(anchor=tk.W, pady=(0, 5))
        
        # Scrolled text widget for multiple URLs
        self.url_text = scrolledtext.ScrolledText(
            url_frame, 
            height=10, 
            font=self.text_font, 
            bg="#ECF0F1", 
            fg="#2C3E50",
            insertbackground="#2C3E50"  # Cursor color
        )
        self.url_text.pack(fill=tk.BOTH, expand=True)
        
        # Options frame
        options_frame = tk.Frame(main_frame, bg="#34495E", padx=10, pady=10, relief=tk.RAISED, bd=1)
        options_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Path selection
        path_frame = tk.Frame(options_frame, bg="#34495E")
        path_frame.pack(fill=tk.X)
        
        path_label = tk.Label(
            path_frame, 
            text="Download Path:", 
            font=self.text_font, 
            bg="#34495E", 
            fg="#ECF0F1"
        )
        path_label.pack(side=tk.LEFT, padx=(0, 5))
        
        self.path_entry = tk.Entry(
            path_frame, 
            font=self.text_font, 
            bg="#ECF0F1", 
            fg="#2C3E50",
            width=50
        )
        self.path_entry.pack(side=tk.LEFT, padx=(0, 5), fill=tk.X, expand=True)
        self.path_entry.insert(0, self.download_path)
        
        browse_button = tk.Button(
            path_frame, 
            text="Browse", 
            font=self.text_font, 
            bg="#3498DB",  # Blue
            fg="#ECF0F1",
            activebackground="#2980B9",  # Darker blue when pressed
            activeforeground="#ECF0F1",
            command=self.browse_path
        )
        browse_button.pack(side=tk.LEFT)
        
        # Format selection
        format_frame = tk.Frame(options_frame, bg="#34495E", pady=10)
        format_frame.pack(fill=tk.X)
        
        format_label = tk.Label(
            format_frame, 
            text="Video Format:", 
            font=self.text_font, 
            bg="#34495E", 
            fg="#ECF0F1"
        )
        format_label.pack(side=tk.LEFT, padx=(0, 5))
        
        self.format_var = tk.StringVar(value="best")
        formats = ["best", "worst", "bestvideo+bestaudio", "bestaudio"]
        
        format_dropdown = ttk.Combobox(
            format_frame, 
            textvariable=self.format_var, 
            values=formats, 
            font=self.text_font,
            width=15
        )
        format_dropdown.pack(side=tk.LEFT)
        
        # Progress frame
        progress_frame = tk.Frame(main_frame, bg="#34495E", padx=10, pady=10, relief=tk.RAISED, bd=1)
        progress_frame.pack(fill=tk.X, pady=(0, 10))
        
        self.progress_var = tk.StringVar(value="Ready to download")
        progress_label = tk.Label(
            progress_frame, 
            textvariable=self.progress_var, 
            font=self.text_font, 
            bg="#34495E", 
            fg="#ECF0F1"
        )
        progress_label.pack(anchor=tk.W)
        
        self.progress_bar = ttk.Progressbar(progress_frame, orient=tk.HORIZONTAL, length=100, mode='indeterminate')
        self.progress_bar.pack(fill=tk.X, pady=(5, 0))
        
        # Buttons frame
        buttons_frame = tk.Frame(main_frame, bg="#2C3E50", pady=10)
        buttons_frame.pack(fill=tk.X)
        
        # Download button
        self.download_button = tk.Button(
            buttons_frame, 
            text="Download Videos", 
            font=self.button_font, 
            bg="#27AE60",  # Green
            fg="#ECF0F1",
            activebackground="#219653",  # Darker green when pressed
            activeforeground="#ECF0F1",
            padx=20, 
            pady=10,
            command=self.download_videos
        )
        self.download_button.pack(side=tk.LEFT, padx=(0, 10))
        
        # Clear button
        clear_button = tk.Button(
            buttons_frame, 
            text="Clear", 
            font=self.button_font, 
            bg="#E74C3C",  # Red
            fg="#ECF0F1",
            activebackground="#C0392B",  # Darker red when pressed
            activeforeground="#ECF0F1",
            padx=20, 
            pady=10,
            command=self.clear_input
        )
        clear_button.pack(side=tk.LEFT)
        
        # Paste button 
        paste_button = tk.Button(
            buttons_frame, 
            text="Paste", 
            font=self.button_font, 
            bg="#F39C12",  # Orange
            fg="#ECF0F1",
            activebackground="#D35400",  # Darker orange when pressed
            activeforeground="#ECF0F1",
            padx=20, 
            pady=10,
            command=self.paste_clipboard
        )
        paste_button.pack(side=tk.RIGHT)
        
        # Style the ttk elements
        style = ttk.Style()
        style.theme_use('clam')
        style.configure("TProgressbar", thickness=10, troughcolor="#34495E", background="#3498DB")
        
        # Status bar
        status_bar = tk.Label(
            self.root, 
            text="Created by YouTube Downloader", 
            bd=1, 
            relief=tk.SUNKEN, 
            anchor=tk.W,
            bg="#1C2833",  # Very dark blue
            fg="#ECF0F1",
            font=("Arial", 8)
        )
        status_bar.pack(side=tk.BOTTOM, fill=tk.X)
    
    def browse_path(self):
        """Open a dialog to select download directory"""
        folder_selected = filedialog.askdirectory()
        if folder_selected:
            self.path_entry.delete(0, tk.END)
            self.path_entry.insert(0, folder_selected)
    
    def paste_clipboard(self):
        """Paste clipboard content into the URL text area"""
        try:
            clipboard_text = self.root.clipboard_get()
            if clipboard_text:
                # If there's already text, add a newline first
                if self.url_text.get("1.0", tk.END).strip():
                    self.url_text.insert(tk.END, "\n")
                self.url_text.insert(tk.END, clipboard_text)
        except tk.TclError:
            # No clipboard content
            pass
    
    def clear_input(self):
        """Clear the URL text area"""
        self.url_text.delete("1.0", tk.END)
        self.progress_var.set("Ready to download")
    
    def download_videos(self):
        """Download videos from the provided URLs"""
        # Get URLs (one per line)
        urls_text = self.url_text.get("1.0", tk.END).strip()
        if not urls_text:
            messagebox.showwarning("No URLs", "Please enter at least one YouTube URL.")
            return
        
        # Split by newlines and filter empty lines
        urls = [url.strip() for url in urls_text.split('\n') if url.strip()]
        
        # Validate URLs
        valid_urls = []
        for url in urls:
            # Simple pattern to validate YouTube URLs
            if re.search(r'(youtube\.com|youtu\.be)', url):
                valid_urls.append(url)
            else:
                self.progress_var.set(f"Invalid URL: {url}")
                messagebox.showwarning("Invalid URL", f"The URL '{url}' does not appear to be a valid YouTube URL.")
                return
        
        # Get download path
        download_path = self.path_entry.get().strip()
        if not download_path:
            messagebox.showwarning("No Path", "Please specify a download path.")
            return
        
        # Create directory if it doesn't exist
        os.makedirs(download_path, exist_ok=True)
        
        # Start download in a separate thread
        self.download_button.config(state=tk.DISABLED)
        self.progress_bar.start(10)
        self.progress_var.set("Downloading...")
        
        download_thread = threading.Thread(
            target=self.download_thread, 
            args=(valid_urls, download_path, self.format_var.get())
        )
        download_thread.daemon = True
        download_thread.start()
    
    def download_thread(self, urls, download_path, format_option):
        """Thread function that performs the download"""
        try:
            total = len(urls)
            for i, url in enumerate(urls):
                self.progress_var.set(f"Downloading {i+1}/{total}: Processing...")
                self.download_youtube_video(url, download_path, format_option)
            
            # Download complete
            self.progress_var.set("Download complete!")
            messagebox.showinfo("Success", "All videos downloaded successfully!")
        except Exception as e:
            # Download failed
            self.progress_var.set(f"Error: {str(e)}")
            messagebox.showerror("Error", f"An error occurred during download:\n{str(e)}")
        finally:
            # Reset UI
            self.root.after(0, self.reset_ui)
    
    def reset_ui(self):
        """Reset UI after download completes"""
        self.download_button.config(state=tk.NORMAL)
        self.progress_bar.stop()
    
    def download_youtube_video(self, video_url, download_path, format_option):
        """Download a YouTube video using yt-dlp"""
        class MyLogger:
            def debug(self, msg):
                if msg.startswith('[download]'):
                    # Update progress message
                    msg = msg.replace('\r', '').strip()
                    if msg:
                        print(msg)  # For debugging
            
            def info(self, msg):
                pass
            
            def warning(self, msg):
                pass
            
            def error(self, msg):
                print(f"Error: {msg}")
        
        ydl_opts = {
            'outtmpl': f'{download_path}/%(title)s.%(ext)s',
            'format': format_option,
            'logger': MyLogger(),
        }
        
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])

if __name__ == "__main__":
    root = tk.Tk()
    app = YouTubeDownloaderGUI(root)
    root.mainloop()

[download] Destination: E:\Python\pythonProject\downloads\Hazrat Yousuf As Ka Sabar ｜｜ Emotional Reminder ｜｜ Shaykh Atif Ahmed.mp4
[download]   0.0% of   15.98MiB at   71.24KiB/s ETA 03:49
[download]   0.0% of   15.98MiB at  186.78KiB/s ETA 01:27
[download]   0.0% of   15.98MiB at  435.82KiB/s ETA 00:37
[download]   0.1% of   15.98MiB at  933.89KiB/s ETA 00:17
[download]   0.2% of   15.98MiB at  636.73KiB/s ETA 00:25
[download]   0.4% of   15.98MiB at  334.91KiB/s ETA 00:48
[download]   0.8% of   15.98MiB at  203.63KiB/s ETA 01:19
[download]   1.6% of   15.98MiB at   87.06KiB/s ETA 03:05
[download]   1.9% of   15.98MiB at   87.00KiB/s ETA 03:04
[download]   2.5% of   15.98MiB at   90.34KiB/s ETA 02:56
[download]   3.1% of   15.98MiB at   96.18KiB/s ETA 02:44
[download]   3.9% of   15.98MiB at   84.61KiB/s ETA 03:05
[download]   4.3% of   15.98MiB at   83.33KiB/s ETA 03:07
[download]   4.7% of   15.98MiB at   68.41KiB/s ETA 03:47
[download]   5.0% of   15.98MiB at   67.16KiB/s ETA 03:51

<h1>With GUI(2)</h1>

In [1]:
import tkinter as tk
from tkinter import ttk, scrolledtext, filedialog, messagebox
import threading
import os
import yt_dlp
from tkinter.font import Font
from tkinter import PhotoImage
import re

class YouTubeDownloaderGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("YouTube Video Downloader")
        self.root.geometry("800x600")
        self.root.configure(bg="#2C3E50")  # Dark blue background
        
        # Default download path
        self.download_path = "E:\\Python\\pythonProject\\downloads"
        
        # Create custom fonts
        self.title_font = Font(family="Arial", size=16, weight="bold")
        self.button_font = Font(family="Arial", size=12, weight="bold")
        self.text_font = Font(family="Arial", size=10)
        
        self.create_widgets()
        
    def create_widgets(self):
        # Main frame
        main_frame = tk.Frame(self.root, bg="#2C3E50", padx=20, pady=20)
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Title
        title_label = tk.Label(
            main_frame, 
            text="YouTube Video Downloader", 
            font=self.title_font, 
            bg="#2C3E50", 
            fg="#ECF0F1"  # Light gray text
        )
        title_label.pack(pady=(0, 20))
        
        # URL input frame
        url_frame = tk.Frame(main_frame, bg="#34495E", padx=10, pady=10, relief=tk.RAISED, bd=1)
        url_frame.pack(fill=tk.BOTH, expand=True, pady=(0, 10))
        
        url_label = tk.Label(
            url_frame, 
            text="Enter YouTube URLs (one per line):", 
            font=self.text_font, 
            bg="#34495E", 
            fg="#ECF0F1"
        )
        url_label.pack(anchor=tk.W, pady=(0, 5))
        
        # Scrolled text widget for multiple URLs
        self.url_text = scrolledtext.ScrolledText(
            url_frame, 
            height=10, 
            font=self.text_font, 
            bg="#ECF0F1", 
            fg="#2C3E50",
            insertbackground="#2C3E50"  # Cursor color
        )
        self.url_text.pack(fill=tk.BOTH, expand=True)
        
        # Options frame
        options_frame = tk.Frame(main_frame, bg="#34495E", padx=10, pady=10, relief=tk.RAISED, bd=1)
        options_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Path selection
        path_frame = tk.Frame(options_frame, bg="#34495E")
        path_frame.pack(fill=tk.X)
        
        path_label = tk.Label(
            path_frame, 
            text="Download Path:", 
            font=self.text_font, 
            bg="#34495E", 
            fg="#ECF0F1"
        )
        path_label.pack(side=tk.LEFT, padx=(0, 5))
        
        self.path_entry = tk.Entry(
            path_frame, 
            font=self.text_font, 
            bg="#ECF0F1", 
            fg="#2C3E50",
            width=50
        )
        self.path_entry.pack(side=tk.LEFT, padx=(0, 5), fill=tk.X, expand=True)
        self.path_entry.insert(0, self.download_path)
        
        browse_button = tk.Button(
            path_frame, 
            text="Browse", 
            font=self.text_font, 
            bg="#3498DB",  # Blue
            fg="#ECF0F1",
            activebackground="#2980B9",  # Darker blue when pressed
            activeforeground="#ECF0F1",
            command=self.browse_path
        )
        browse_button.pack(side=tk.LEFT)
        
        # Format selection
        format_frame = tk.Frame(options_frame, bg="#34495E", pady=10)
        format_frame.pack(fill=tk.X)
        
        format_label = tk.Label(
            format_frame, 
            text="Video Format:", 
            font=self.text_font, 
            bg="#34495E", 
            fg="#ECF0F1"
        )
        format_label.pack(side=tk.LEFT, padx=(0, 5))
        
        self.format_var = tk.StringVar(value="best")
        formats = ["best", "worst", "bestvideo+bestaudio", "bestaudio"]
        
        format_dropdown = ttk.Combobox(
            format_frame, 
            textvariable=self.format_var, 
            values=formats, 
            font=self.text_font,
            width=15
        )
        format_dropdown.pack(side=tk.LEFT)
        
        # Progress frame
        progress_frame = tk.Frame(main_frame, bg="#34495E", padx=10, pady=10, relief=tk.RAISED, bd=1)
        progress_frame.pack(fill=tk.X, pady=(0, 10))
        
        self.progress_var = tk.StringVar(value="Ready to download")
        progress_label = tk.Label(
            progress_frame, 
            textvariable=self.progress_var, 
            font=self.text_font, 
            bg="#34495E", 
            fg="#ECF0F1"
        )
        progress_label.pack(anchor=tk.W)
        
        # Changed to determinate mode for percentage display
        self.progress_bar = ttk.Progressbar(progress_frame, orient=tk.HORIZONTAL, length=100, mode='determinate')
        self.progress_bar.pack(fill=tk.X, pady=(5, 0))
        
        # Buttons frame
        buttons_frame = tk.Frame(main_frame, bg="#2C3E50", pady=10)
        buttons_frame.pack(fill=tk.X)
        
        # Download button
        self.download_button = tk.Button(
            buttons_frame, 
            text="Download Videos", 
            font=self.button_font, 
            bg="#27AE60",  # Green
            fg="#ECF0F1",
            activebackground="#219653",  # Darker green when pressed
            activeforeground="#ECF0F1",
            padx=20, 
            pady=10,
            command=self.download_videos
        )
        self.download_button.pack(side=tk.LEFT, padx=(0, 10))
        
        # Clear button
        clear_button = tk.Button(
            buttons_frame, 
            text="Clear", 
            font=self.button_font, 
            bg="#E74C3C",  # Red
            fg="#ECF0F1",
            activebackground="#C0392B",  # Darker red when pressed
            activeforeground="#ECF0F1",
            padx=20, 
            pady=10,
            command=self.clear_input
        )
        clear_button.pack(side=tk.LEFT)
        
        # Paste button 
        paste_button = tk.Button(
            buttons_frame, 
            text="Paste", 
            font=self.button_font, 
            bg="#F39C12",  # Orange
            fg="#ECF0F1",
            activebackground="#D35400",  # Darker orange when pressed
            activeforeground="#ECF0F1",
            padx=20, 
            pady=10,
            command=self.paste_clipboard
        )
        paste_button.pack(side=tk.RIGHT)
        
        # Style the ttk elements
        style = ttk.Style()
        style.theme_use('clam')
        style.configure("TProgressbar", thickness=10, troughcolor="#34495E", background="#3498DB")
        
        # Status bar
        status_bar = tk.Label(
            self.root, 
            text="Created by YouTube Downloader", 
            bd=1, 
            relief=tk.SUNKEN, 
            anchor=tk.W,
            bg="#1C2833",  # Very dark blue
            fg="#ECF0F1",
            font=("Arial", 8)
        )
        status_bar.pack(side=tk.BOTTOM, fill=tk.X)
    
    def browse_path(self):
        """Open a dialog to select download directory"""
        folder_selected = filedialog.askdirectory()
        if folder_selected:
            self.path_entry.delete(0, tk.END)
            self.path_entry.insert(0, folder_selected)
    
    def paste_clipboard(self):
        """Paste clipboard content into the URL text area"""
        try:
            clipboard_text = self.root.clipboard_get()
            if clipboard_text:
                # If there's already text, add a newline first
                if self.url_text.get("1.0", tk.END).strip():
                    self.url_text.insert(tk.END, "\n")
                self.url_text.insert(tk.END, clipboard_text)
        except tk.TclError:
            # No clipboard content
            pass
    
    def clear_input(self):
        """Clear the URL text area"""
        self.url_text.delete("1.0", tk.END)
        self.progress_var.set("Ready to download")
        self.progress_bar["value"] = 0
    
    def download_videos(self):
        """Download videos from the provided URLs"""
        # Get URLs (one per line)
        urls_text = self.url_text.get("1.0", tk.END).strip()
        if not urls_text:
            messagebox.showwarning("No URLs", "Please enter at least one YouTube URL.")
            return
        
        # Split by newlines and filter empty lines
        urls = [url.strip() for url in urls_text.split('\n') if url.strip()]
        
        # Validate URLs
        valid_urls = []
        for url in urls:
            # Simple pattern to validate YouTube URLs
            if re.search(r'(youtube\.com|youtu\.be)', url):
                valid_urls.append(url)
            else:
                self.progress_var.set(f"Invalid URL: {url}")
                messagebox.showwarning("Invalid URL", f"The URL '{url}' does not appear to be a valid YouTube URL.")
                return
        
        # Get download path
        download_path = self.path_entry.get().strip()
        if not download_path:
            messagebox.showwarning("No Path", "Please specify a download path.")
            return
        
        # Create directory if it doesn't exist
        os.makedirs(download_path, exist_ok=True)
        
        # Start download in a separate thread
        self.download_button.config(state=tk.DISABLED)
        self.progress_bar["value"] = 0
        self.progress_var.set("Downloading...")
        
        download_thread = threading.Thread(
            target=self.download_thread, 
            args=(valid_urls, download_path, self.format_var.get())
        )
        download_thread.daemon = True
        download_thread.start()
    
    def download_thread(self, urls, download_path, format_option):
        """Thread function that performs the download"""
        try:
            total = len(urls)
            for i, url in enumerate(urls):
                # Update progress for overall video count
                overall_progress = (i / total) * 100
                self.root.after(0, lambda p=overall_progress: self.update_progress(p))
                self.root.after(0, lambda msg=f"Downloading {i+1}/{total} videos": self.update_status(msg))
                
                self.download_youtube_video(url, download_path, format_option)
            
            # Download complete
            self.root.after(0, lambda: self.update_progress(100))
            self.root.after(0, lambda: self.update_status("Download complete!"))
            messagebox.showinfo("Success", "All videos downloaded successfully!")
        except Exception as e:
            # Download failed
            self.root.after(0, lambda msg=f"Error: {str(e)}": self.update_status(msg))
            messagebox.showerror("Error", f"An error occurred during download:\n{str(e)}")
        finally:
            # Reset UI
            self.root.after(0, self.reset_ui)
    
    def update_progress(self, value):
        """Update progress bar value"""
        self.progress_bar["value"] = value
    
    def update_status(self, message):
        """Update status message"""
        self.progress_var.set(message)
    
    def reset_ui(self):
        """Reset UI after download completes"""
        self.download_button.config(state=tk.NORMAL)
    
    def download_youtube_video(self, video_url, download_path, format_option):
        """Download a YouTube video using yt-dlp"""
        # Create a progress hook to update the UI
        def my_hook(d):
            if d['status'] == 'downloading':
                # Calculate percentage
                if 'total_bytes' in d and d['total_bytes'] > 0:
                    percentage = (d['downloaded_bytes'] / d['total_bytes']) * 100
                    self.root.after(0, lambda p=percentage: self.update_progress(p))
                    self.root.after(0, lambda msg=f"Downloading: {d['filename'].split('/')[-1]} - {percentage:.1f}%": 
                                  self.update_status(msg))
                elif 'downloaded_bytes' in d:
                    # If we don't know the total size, just show downloaded amount
                    downloaded_mb = d['downloaded_bytes'] / 1024 / 1024
                    self.root.after(0, lambda msg=f"Downloading: {downloaded_mb:.1f} MB (unknown size)": 
                                  self.update_status(msg))
            elif d['status'] == 'finished':
                self.root.after(0, lambda: self.update_status(f"Download finished: {d['filename'].split('/')[-1]}"))
                self.root.after(0, lambda: self.update_progress(100))
            elif d['status'] == 'error':
                self.root.after(0, lambda: self.update_status(f"Error downloading: {d.get('filename', '')}"))
        
        ydl_opts = {
            'outtmpl': f'{download_path}/%(title)s.%(ext)s',
            'format': format_option,
            'progress_hooks': [my_hook],
        }
        
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])

if __name__ == "__main__":
    root = tk.Tk()
    app = YouTubeDownloaderGUI(root)
    root.mainloop()